# introduction

지난번의 general encoder-decoder model에서는 '정보 압박' 을 줄이기 위해서 디코더의 매 시점마다 맥락 벡터 $z$와 인풋 단어 $y_t$그리고 은닉상태 $s_t$를 선형함수 $f$에 넣어 예측값을 구하였다.

compression을 줄였다고는 하지만, 맥락 벡터가 여전히 모든 source 문장의 정보를 담고있지는 못하였기 때문에 이번논문에서 attention을 사용함으로써 이를 향상시킨다.

attention은 일단 어텐션 벡터인 $a$를 계산해야하는데 이는 0과 1사이의 값이며 전체 합이 1이 되어야한다. 그다음에 source 문장의 은닉 상태 $H$의 가중치의 합을 구해서 weighted source vector인 $w$를 구할 수 있다.

$$w = \sum_{i}a_ih_i$$

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import time

In [3]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [5]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [8]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)


In [42]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [18]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [21]:
for idx,val in enumerate(train_iterator) :
    src = val.src
    trg = val.trg
    break

# Encoder

기존 모델에서 단일 GRU레이어를 썼던 것과는 달리 인코더에서는 양방향의 RNN을 사용한다. 정방향과 역방향 RNN, 코드로는 bidirectional =True 만해주면됨


$$\begin{align*}
h_t^\rightarrow = \text{EncoderGRU}^\rightarrow(x_t^\rightarrow,h_t^\rightarrow)\\
h_t^\leftarrow = \text{EncoderGRU}^\leftarrow(x_t^\leftarrow,h_t^\leftarrow)
\end{align*}$$

RNN은 output과 hidden을 반환한다.
output은 [src sent len, batch size, hid dim * num directions]으로 구성되어 있다.

$h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$

In [81]:
len(SRC.vocab)

7853

In [93]:
SRC.vocab

In [120]:
input_dim = len(SRC.vocab)
emb_dim = 256
enc_hid_dim = 512
dec_hid_dim = 512
dropout = 0.5
batch_size = 128
embedding = nn.Embedding(input_dim,emb_dim, padding_idx = 1).to(device)
rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True).to(device)
#여기서 bidirection이니깐 hid_dim*2로들어감 
fc = nn.Linear(enc_hid_dim *2 , )


In [121]:
input_dim

7853

In [123]:
input_dim = len(SRC.vocab) #SRC 토큰의 수
emb_dim = 256
enc_hid_dim = 512
dec_hid_dim = 512
dropout = 0.5
batch_size = 128

embedding = nn.Embedding(input_dim, emb_dim,padding_idx=1).to(device)
#7853x7853에서 256차원으로 축약해서 임베딩시키면서 sementic하게

rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True).to(device)
#RNN에 emb_dim,hid_dim이 들어가는데 양방향

fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim).to(device)

dropout = nn.Dropout(dropout)

embedded = dropout(embedding(src))

outputs, hidden = rnn(embedded)

hidden = torch.tanh(fc(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),
                                dim = 1)))

In [127]:
output_dim

5893

In [83]:
input_dim

7853

In [94]:
embedding #7853개의 token을 패딩포함하여 256차원으로 임베딩시킨다

Embedding(7853, 256, padding_idx=1)

In [95]:
rnn #임베딩된 input을 rnn에 넣는데 hid_dim이 512이면서 두개씩

GRU(256, 512, bidirectional=True)

In [96]:
embedded = dropout(embedding(src))
print(embedded.shape) #길이제한이 33, 배치사이즈128(문장128개씩)
outputs, hidden = rnn(embedded)
print(outputs.shape,hidden.shape)

torch.Size([33, 128, 256])
torch.Size([33, 128, 1024]) torch.Size([2, 128, 512])


In [109]:
hidden

tensor([[[-0.0188,  0.0123,  0.0134,  ..., -0.0333,  0.0043, -0.0486],
         [-0.0186,  0.0134,  0.0119,  ..., -0.0332,  0.0045, -0.0498],
         [-0.0192,  0.0123,  0.0134,  ..., -0.0337,  0.0040, -0.0487],
         ...,
         [-0.0192,  0.0123,  0.0133,  ..., -0.0338,  0.0041, -0.0487],
         [-0.0191,  0.0123,  0.0134,  ..., -0.0336,  0.0039, -0.0486],
         [-0.0195,  0.0112,  0.0122,  ..., -0.0326,  0.0016, -0.0515]],

        [[ 0.3103,  0.4873, -0.0899,  ..., -0.1195, -0.2087, -0.0634],
         [ 0.0239,  0.1473,  0.0405,  ..., -0.3432,  0.1152,  0.2024],
         [-0.1139,  0.5564, -0.5278,  ..., -0.1089, -0.0396, -0.1027],
         ...,
         [-0.0586, -0.0999,  0.4446,  ..., -0.0928, -0.3009,  0.3099],
         [ 0.1354,  0.0922, -0.2902,  ..., -0.4236,  0.3173, -0.2309],
         [ 0.0095,  0.2651, -0.1910,  ...,  0.3093, -0.4063, -0.2340]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)

$y_{t+1}$을 예측하는데 있어서 source sentence에 있어서 어떤 word가 가장 많은 attention을 주느냐를 파악하기위해 기존 인코더의 양방향 hidden states와 decoder의 전시점 hidden이 필요

In [117]:
     
embedding = nn.Embedding(output_dim, emb_dim,padding_idx=1)
hidden_dim = hidden_dim
output_dim = output_dim
num_layers = num_layers        
rnn = nn.GRU(emb_dim+hidden_dim*2,hidden_dim*2, num_layers=num_layers, \
                      dropout=dropout,batch_first=True)    
dropout = nn.Dropout(dropout)
fully_connect = nn.Linear(hidden_dim*2,output_dim)
attention_obj = attention_obj
batch_size = batch_size



NameError: name 'hidden_dim' is not defined